In [19]:
import copy

from dataclasses import dataclass
from typing import List
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rnd
import re

from alns import ALNS, State
from alns.criteria import HillClimbing
from alns.weight_schemes import SimpleWeights

In [20]:
%matplotlib inline

In [21]:
SEED = 5432

# The resource-constrained project scheduling problem

The following explanation is largely based on [this paper](https://pms2020.sciencesconf.org/300164/document).

The goal of the RCPSP is to schedule a set of project activities $V = \{ 0, 1, 2, \ldots, n \}$, such that the makespan of the project is minimised.
Each activity $i \in V$ has a duration $d_i \in \mathbb{N}$.
Precedence constraints impose that an activity $i \in V$ can only start after all its predecessor activities have been completed.
The precedence constraints are given by a set of edges $E \subset V \times V$, where $(i, j) \in E$ means that $i$ must be completed before $j$ can commence.
Resource constraints, on the other hand, impose that an activity can only be scheduled if sufficient resources are available.
There are $K = \{ 1, 2, \ldots, m \}$ renewable resources available, with $R_k$ indicating the availability of resource $k$.
Each activity $i \in V$ requires $r_{ik}$ units of resource $k$.
A solution to the RCPSP is a schedule of activities $S = \{ S_0, S_1, \ldots, S_n \}$, where $S_i$ is the starting time of activity $i$.
The project starts at time $S_0 = 0$, and completes at $S_n$, where activities $0$ and $n$ are dummy activities that represent the start and completion of the project, respectively.

In this notebook, we solve an instance of the RCPSP using ALNS.
In particular, we solve instance `j9041_6` of the [PSPLib](http://www.om-db.wi.tum.de/psplib/library.html) benchmark suite.
This instance consists of 90 jobs, and four resources.


## Data instance

In [32]:
@dataclass
class ProblemData:
    num_jobs: int
    num_resources: int

    durations: List[int]  # job durations
    successors: List[List[int]]  # job successors
    needs: List[List[int]]  # job resource needs
    resources: List[int]  # resource capacities

    @property
    def last_job(self) -> int:
        return self.num_jobs - 1

    @classmethod
    def read_instance(cls, path: str) -> "ProblemData":
        """
        Reads an instance of the RCPSP from a file.
        Assumes the data is in the PSPLib format.

        Loosely based on:
        https://github.com/baobabsoluciones/hackathonbaobab2020.
        """
        with open(path) as fh:
            lines = fh.readlines()

        prec_idx = lines.index("PRECEDENCE RELATIONS:\n")
        req_idx = lines.index("REQUESTS/DURATIONS:\n")
        avail_idx = lines.index("RESOURCEAVAILABILITIES:\n")

        successors = []

        for line in lines[prec_idx + 2: req_idx - 1]:
            _, _, modes, num_succ, *jobs, _ = re.split("\s+", line)
            successors.append(list(map(int, jobs)))

        needs = []
        durations = []

        for line in lines[req_idx + 3: avail_idx - 1]:
            _, _, _, duration, *consumption, _ = re.split("\s+", line)

            needs.append(list(map(int, consumption)))
            durations.append(int(duration))

        _, *avail, _ = re.split("\s+", lines[avail_idx + 2])
        resources = list(map(int, avail))

        return ProblemData(len(durations),
                           len(resources),
                           durations,
                           successors,
                           needs,
                           resources)

In [34]:
instance = ProblemData.read_instance('j9041_6.sm')

## Solution state

In [36]:
@dataclass
class RcpspState(State):
    """
    Solution state for the resource-constrained project scheduling problem.
    """
    _schedule: List[int]
    _UNSCHEDULED = object()

    @property
    def unscheduled(self) -> List[int]:
        return [idx for idx in range(instance.num_jobs)
                if self._schedule[idx] is self._UNSCHEDULED]

    def objective(self) -> int:
        return self._schedule[instance.last_job]

    def unschedule(self, job: int):
        self._schedule[job] = self._UNSCHEDULED

    def schedule(self, job: int, start: int):
        self._schedule[job] = start

    def plot(self):
        fig = plt.figure(figsize=(12, 6 + instance.num_resources))

        hr = [1] * (instance.num_resources + 1)
        hr[0] = 6

        gs = plt.GridSpec(nrows=1 + instance.num_resources,
                          ncols=1,
                          height_ratios=hr)

        gantt = fig.add_subplot(gs[0, 0])
        # TODO

        for res in range(instance.num_resources):
            res_ax = fig.add_subplot(gs[res + 1, 0], sharex=gantt)
            # TODO

## Destroy operators

## Repair operators

## Initial solution

## Heuristic solution